In [12]:
import numpy as np

In [13]:
with open('data/input_22.txt') as fh:
    file_input = fh.read().strip()

In [14]:
file_input

'depth: 10689\ntarget: 11,722'

In [218]:
# real input
depth = 10689
target = (11, 722)
nx, ny = target
nx += 700
ny += 20

In [119]:
# test
depth = 510
target = (10, 10)


In [232]:
el = np.zeros((nx, ny), dtype='uint64')
el[:, 0] = get_el(np.arange(nx) * 16807)
el[0, :] = get_el(np.arange(ny) * 48271)

for i in range(1, nx):
    for j in range(1, ny):
        if i == target[0] and j == target[1]:
            el[i, j] = get_el(0)

            continue
        el[i, j] = get_el(el[i-1, j] * el[i, j-1]) # % 20183

In [233]:
cave = el % 3

In [234]:
cave.T

array([[0, 2, 1, ..., 2, 0, 2],
       [0, 2, 0, ..., 2, 0, 0],
       [1, 2, 1, ..., 0, 2, 2],
       ...,
       [1, 0, 0, ..., 1, 1, 2],
       [2, 1, 2, ..., 2, 2, 2],
       [2, 0, 1, ..., 1, 0, 2]], dtype=uint64)

Part A

In [223]:
cave[:target[0]+1, :target[1]+1].sum()

8575

Part B

In [230]:
def dist_target(node, target=target):
    return abs(node[0]-target[0]) + abs(node[1]-target[1])

def is_valid(ctype, eq):
    if ctype == 0:
        if eq == 1 or eq == 2:
            return True
        return False
    elif ctype == 1:
        if eq == 0 or eq == 2:
            return True
        return False
    elif ctype == 2:
        if eq == 1 or eq == 0:
            return True
        return False
    raise

def get_neighbors(node, cave=cave):
    x, y, eq = node
    if x > 0 and is_valid(eq, cave[x-1, y]):
        yield (x-1, y, eq), 1
    if y > 0 and is_valid(eq, cave[x, y-1]):
        yield (x, y-1, eq), 1
    if is_valid(eq, cave[x+1, y]):
        yield (x+1, y, eq), 1
    if is_valid(eq, cave[x, y+1]):
        yield (x, y+1, eq), 1
    for i in range(3):
        if i != eq and is_valid(i, cave[x, y]):
            yield(x, y, i), 7
            
def recontruct_path(cameFrom, goal, start):
    next_step = cameFrom[goal]
    path = [next_step]
    while next_step != start:
        next_step = cameFrom[next_step]
        path.append(next_step)
    return path

In [225]:
from collections import defaultdict, deque
def get_path(cave=cave, goal=(0,0,1)):
    gScore = defaultdict(lambda: np.inf)
    fScore = defaultdict(lambda: np.inf)
    visited = set([])
    start = (0, 0, 1)
    gScore[start] = 0
    fScore[start] = dist_target(start)
    cameFrom = {}
    stack = [start]
    while stack:
        stack.sort(key=fScore.get, reverse=True)
        current = stack.pop()
        if current == goal:
            return recontruct_path(cameFrom, goal, start), gScore
        visited.add(current)

        for nb, dt in get_neighbors(current, cave=cave):
            if nb in visited:
                continue

            tmp_gScore = gScore[current] + dt
            if nb not in stack:
                stack.append(nb)
            elif tmp_gScore >= gScore[nb]:
                continue
            cameFrom[nb] = current
            gScore[nb] = tmp_gScore
            fScore[nb] = gScore[nb] + dist_target(nb)
            

In [254]:
# from collections import defaultdict, deque
# import heapq
# def get_path_heapq(cave=cave, goal=(0,0,1)):
#     gScore = defaultdict(lambda: np.inf)
#     fScore = defaultdict(lambda: np.inf)
#     visited = set([])
#     start = (0, (0, 0, 1))
#     gScore[start] = 0
#     # fScore[start] = dist_target(start)
#     cameFrom = {}
#     stack = []
#     heapq.heappush(stack, start)
#     while stack:
#         minutes, current = heapq.heappop(stack)
#         if current == goal:
#             return recontruct_path(cameFrom, goal, start), gScore
#         if current in visited:
#             continue
#         if gScore[current] <= minutes:
#             continue
#         visited.add(current)

#         for nb, dt in get_neighbors(current, cave=cave):
#             if nb in visited:
#                 continue

#             tmp_gScore = gScore[current] + dt
#             heapq.heappush(stack, (minutes+dt, nb))           
#             if tmp_gScore >= gScore[nb]:
#                 continue
#             cameFrom[nb] = current
#             gScore[nb] = tmp_gScore
#             fScore[nb] = gScore[nb] + dist_target(nb)

In [257]:
%%time
goal = (target[0], target[1], 1)
path, score = get_path(cave=cave, goal=goal)

CPU times: user 1min 21s, sys: 2.29 s, total: 1min 23s
Wall time: 1min 31s


Part B

In [228]:
score[goal]

999